In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
 
model = nn.Sequential(
    nn.Linear(2, 10),
    nn.ReLU(),
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

# Define the loss function
loss_fn = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np 

s1d1df = pd.read_csv('s1d1pings.csv')
s1d1df['TIMESTAMP'] = pd.to_datetime(s1d1df['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S.%f')
s1d1df.reset_index

s1d1df['date'] = pd.to_datetime(s1d1df['date'], format='%Y-%m-%d')
s1d1df['day_of_week'] = s1d1df['date'].dt.day_name()
s1d1df['day_of_week'] = s1d1df['date'].dt.day_name()
day_to_number = {
    'Sunday': 0.1,
    'Monday': 0.2,
    'Tuesday': 0.3,
    'Wednesday': 0.4,
    'Thursday': 0.5,
    'Friday': 0.6,
    'Saturday': 0.7
}
s1d1df['day_of_week_number'] = s1d1df['day_of_week'].map(day_to_number)

s1d1df['normalized_time'] = (s1d1df['TIMESTAMP'].dt.hour * 3600 + s1d1df['TIMESTAMP'].dt.minute * 60 + s1d1df['TIMESTAMP'].dt.second) / 86400
pd.set_option('display.float_format', '{:.8f}'.format)
s1d1df

,TIMESTAMP,ping,date,hour,day_of_week,day_of_week_number,normalized_time
0,2024-03-22 12:55:17.235189,58.00000000,2024-03-22,12,Friday,0.60000000,0.53839120
1,2024-03-22 12:55:17.235189,56.00000000,2024-03-22,12,Friday,0.60000000,0.53839120
2,2024-03-22 12:55:17.235189,54.00000000,2024-03-22,12,Friday,0.60000000,0.53839120
3,2024-03-22 12:55:17.235189,53.00000000,2024-03-22,12,Friday,0.60000000,0.53839120
4,2024-03-22 12:55:17.235189,53.00000000,2024-03-22,12,Friday,0.60000000,0.53839120
...,...,...,...,...,...,...,...
6406,2024-04-09 06:42:12.659485,44.00000000,2024-04-09,6,Tuesday,0.30000000,0.27930556
6407,2024-04-09 07:16:19.492482,8.00000000,2024-04-09,7,Tuesday,0.30000000,0.30299769
6408,2024-04-09 07:16:19.492482,51.00000000,2024-04-09,7,Tuesday,0.30000000,0.30299769
6409,2024-04-09 08:07:29.063338,6.00000000,2024-04-09,8,Tuesday,0.30000000,0.33853009


In [3]:
test_df = s1d1df.sample(frac=0.2, random_state=2) 
training_df = s1d1df.drop(test_df.index)
test_df
training_df

x_train = training_df[['normalized_time', 'day_of_week_number']].values  # Features
y_train = training_df['ping'].values/100  # Labels
# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test = test_df[['normalized_time', 'day_of_week_number']].values  # Features
y_test = test_df['ping'].values/100  # Labels
# Convert to PyTorch tensors
x_test_tensor = torch.tensor(x_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_train, dtype=torch.float32)

In [4]:
num_epochs = 300
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    optimizer.zero_grad()  # Clear gradients from previous iteration

    # Forward pass
    y_pred = model(x_train_tensor)

    # Compute loss
    loss = loss_fn(y_pred.squeeze(), y_train_tensor)  # Squeeze y_pred to match y_train_tensor shape
    print(loss.item())

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

#print(model)
print(y_pred)
print(y_train_tensor)
# Compute validation loss
val_loss = loss_fn(y_pred.squeeze(), y_train_tensor)
    
# Print validation loss (optional)
print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {val_loss.item():.4f}')

0.3124854266643524
0.3054186999797821
0.298404723405838
0.291433185338974
0.28450456261634827
0.2776572108268738
0.2708943784236908
0.26421841979026794
0.2576313614845276
0.2511352002620697
0.24473172426223755
0.23842264711856842
0.23220953345298767
0.22609378397464752
0.2200765609741211
0.21415923535823822
0.20834289491176605
0.20262877643108368
0.19701765477657318
0.19151082634925842
0.18610879778862
0.1808125525712967
0.17562271654605865
0.17053991556167603
0.165564626455307
0.16069746017456055
0.15593869984149933
0.15128874778747559
0.14674785733222961
0.14231613278388977
0.13799376785755157
0.13378076255321503
0.12967704236507416
0.12568256258964539
0.12179713696241379
0.11802033334970474
0.11435201019048691
0.1107916459441185
0.10733877867460251
0.10399282723665237
0.10075312107801437
0.09761886298656464
0.09458926320075989
0.0916634202003479
0.08884046226739883
0.08611937612295151
0.08349922299385071
0.08097866177558899
0.0785563737154007
0.07623110711574554
0.0740015059709549
0

In [5]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    # Forward pass on validation data
    y_pred_val = model(x_test_tensor)
    
    # Compute validation loss
    val_loss = loss_fn(y_pred_val.squeeze(), y_test_tensor)
    
    # Print validation loss (optional)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}')

x_test = x_test_tensor.detach().numpy()
y_test = y_test_tensor.detach().numpy()

df = pd.DataFrame({
    'x_test_1': x_test_tensor[:, 0].flatten(),  # Assuming x_test has 2 columns, showing the first one
    'x_test_2': x_test_tensor[:, 1].flatten(),  # Assuming x_test has 2 columns, showing the second one
    'y_test': y_test_tensor.flatten(),
    'y_pred': y_pred_val.flatten()

})
df

Epoch [300/300], Validation Loss: 0.0326


,x_test_1,x_test_2,y_test,y_pred
0,0.53839123,0.60000002,0.57999998,0.51116395
1,0.53839123,0.60000002,0.56000000,0.51116395
2,0.53839123,0.60000002,0.54000002,0.51116395
3,0.53839123,0.60000002,0.52999997,0.51116395
4,0.53975695,0.60000002,0.57999998,0.51133823
...,...,...,...,...
5124,0.25528935,0.30000001,0.73000002,0.43282348
5125,0.27930555,0.30000001,0.10000000,0.43588755
5126,0.27930555,0.30000001,0.44000000,0.43588755
5127,0.30299768,0.30000001,0.08000000,0.43891025
